In [48]:
from numpy import *
from os import listdir
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold #交叉验证
from sklearn.model_selection import GridSearchCV  #通过网络方式来获取参数
from sklearn.model_selection import train_test_split #将数据集分开成训练集和测试集

from sklearn.naive_bayes import MultinomialNB

In [49]:
def img2vector(filename):
    returnVect = zeros((1, 1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0, 32 * i + j] = int(lineStr[j])
    return returnVect

In [56]:
def handwritingClassTest():
    # 1. 导入训练数据
    hwLabels = []
    trainingFileList = listdir('trainingDigits')  # load the training set
    m = len(trainingFileList)
    trainingMat = zeros((m, 1024))
    # hwLabels存储0～9对应的index位置， trainingMat存放的每个位置对应的图片向量
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]  # take off .txt
        classNumStr = int(fileStr.split('_')[0])
        hwLabels.append(classNumStr)
        # 将 32*32的矩阵->1*1024的矩阵
        trainingMat[i, :] = img2vector('trainingDigits/%s' % fileNameStr)    
        
    print(trainingMat.shape,len(hwLabels))   
       
    '''
    #K折交叉验证
    seed = 7 #重现随机生成的训练
    test_size = 0.1 #80%测试，20%训练
    X_train, X_test, Y_train, Y_test = train_test_split(trainingMat, hwLabels, 
                                                        test_size=test_size, 
                                                        random_state=seed)
    #KNN = KNeighborsClassifier()  
    MNB = MultinomialNB()
    
    #param_grid = {'n_neighbors':range(1,15,2)} 
    param_grid = {'alpha':[i/100.0 for i in range(200)] }
    
    kflod = StratifiedKFold(n_splits = 10, shuffle = True,random_state = seed)#将训练/测试数据集划分10个互斥子集， 
    
    #grid_search = GridSearchCV(KNN,param_grid,n_jobs = -1,cv = kflod)
    
    grid_search = GridSearchCV(MNB,param_grid,n_jobs = -1,cv = kflod)
    
    grid_result = grid_search.fit(X_train, Y_train) #运行网格搜索
    print("Best: %f using %s" % (grid_result.best_score_,grid_search.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    params = grid_result.cv_results_['params']
    for mean,param in zip(means,params):
        print("%f  with:   %r" % (mean,param))
    '''
    #K折交叉验证得最优K值为3：
    #KNN = KNeighborsClassifier(3)
    #KNN.fit(trainingMat,hwLabels)
    
    #Kz折交叉验证得最优alpha值为1.41-1.67：
    MNB = MultinomialNB(alpha=1.5)
    MNB.fit(trainingMat,hwLabels)
    
    
    # 2. 导入测试数据
    testFileList = listdir('testDigits')  # iterate through the test set
    errorCount = 0.0
    mTest = len(testFileList)
    for i in range(mTest):
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]  # take off .txt
        classNumStr = int(fileStr.split('_')[0])
        vectorUnderTest = img2vector('testDigits/%s' % fileNameStr)
        
          
        #获得测试结果
        '''
        classifierResult = KNN.predict(vectorUnderTest)
        #print("the classifier came back with: %d, the real answer is: %d" % (classifierResult, classNumStr))
        if (classifierResult != classNumStr): 
            errorCount += 1.0
        '''   

        classifierResult = MNB.predict(vectorUnderTest)
        if (classifierResult != classNumStr): 
            errorCount += 1.0                    
        
    print("\nthe total number of errors is: %d" % errorCount)
    print("\nthe total error rate is: %f" % (errorCount / float(mTest)))    


In [57]:
handwritingClassTest()

(1934, 1024) 1934

the total number of errors is: 72

the total error rate is: 0.076110


## KNN和NB的比较：
### 从分类表现来看，KNN准确率高，速度较慢；MNB准确率低，速度快。
### 超参调优方面看，KNN超参依赖样本，NB的超参只关于本身。有啥好处现在也说不清楚……